#  A little introduction to Keras, using TensorFlow
    
 Keras is a simple and high-level definition interface 
 that make the use of TensorFlow more easier<br><br>

<font color=red size=2 ><B><i>!! This tutorial assumes that you have configured Keras
 to use the TensorFlow backend</font><B></i></br><br>





    
## Part One : Calling Keras layers on TensorFlow tensors

 __We will start with a simple example of MNIST digits classification.<br>
 We will build a TensorFlow digits classifier using a stack of Keras Dense layers__
 

***

We first have to create a Tensorflow session and registering it with Keras.<br>
* Keras will use this session to initialize all internal variables

In [1]:
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

Using TensorFlow backend.


We can now building our classifier exactly like we would do with TensorFlow

In [2]:
# placeholder that will contain our input digits, as flat vectors
img = tf.placeholder(tf.float32, shape=(None, 784))

 Keras layers can be called on TensorFlow tensors :<br><br>
 *Using Keras layers will speed up the model definition process*

In [3]:
from keras.layers import Dense

# Fully-connected layer with 128 unis
# and ReLU activation (Rectified Linear Unit)
x = Dense(128, activation='relu')(img)
x = Dense(128, activation='relu')(x)

# Output layer with 10 units and a softmax activation
preds = Dense(10, activation='softmax')(x)

We will define the placeholder for the labels and the loss function to use

In [4]:
labels = tf.placeholder(tf.float32, shape=(None, 10))

from keras.objectives import categorical_crossentropy
loss = tf.reduce_mean(categorical_crossentropy(labels, preds))

Now the training part, with a TensorFlow optimizer :

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

#Initialization of all variables
init_op = tf.global_variables_initializer()
sess.run(init_op)

#Run training loop
with sess.as_default():
    for i in range(100):
        batch = mnist_data.train.next_batch(50)
        train_step.run(feed_dict={img: batch[0], labels: batch[1]})

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


Now we can evaluate the model :

In [6]:
from keras.metrics import categorical_accuracy as accuracy
acc_value = accuracy(labels, preds)
with sess.as_default():
    print (acc_value.eval(feed_dict={img: mnist_data.test.images,
                                    labels: mnist_data.test.labels}))

[ 1.  1.  1. ...,  0.  0.  1.]


Here we have used Keras as a syntaxical shortcut to generate an operator that just map some input tensors to some output tensors.
The optimization is done via a TensorFlow native optimizer rather than a Keras one and no Keras model have been used.<br><br>

Concerning the optimizers performances, there is little speed differences between Keras and TensorFlow use. Keras seems to be faster than TensorFlow nativ optimizer on most cases, but the differences is only from 5 to 10%.
So for most of the projects, it doesn't really matter which one you choose.



### Some different behaviors during training and testing phases

Some Keras layers (like Dropout or BatchNormalization) have different behaviors at training time and testing time.<br>
It's possible to know if a layer uses the "leaarning phase" (train/test) by printin the layer.uses_learning_phase variable. A boolean that is set to True if the layer has a different behavior on training mode and testin mode, and False otherwise.<br><br>

If the model use such layers, you need to specify the value of the learning phase as part of feed_dict. That way the model knows wether to apply the layer.<br><br>

The Keras learnin phase is accessible via the Keras backend. <br>
Like this :

In [7]:
from keras import backend as K
print (K.learning_phase())

Tensor("keras_learning_phase:0", dtype=bool)


To set the use of learning phase, write the value 1 for training mode or 0 for testing mode, to feed_dict :<br>
*(here we have added "session=sess" because run() need a default session to work)*

In [8]:
# train mode
train_step.run(feed_dict={img: batch[0], labels: batch[1],
                          K.learning_phase(): 1}, session=sess)

Here's the code to add the Dropout layer to our previous MNIST example :

In [9]:
from keras.layers import Dropout
from keras import backend as K

img = tf.placeholder(tf.float32, shape=(None, 784))
labels = tf.placeholder(tf.float32, shape=(None, 10))

x = Dense(128, activation='relu')(img)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
preds = Dense(10, activation='softmax')(x)

loss = tf.reduce_mean(categorical_crossentropy(labels, preds))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# Initialize all variables
init_op = tf.global_variables_initializer()

sess.run(init_op)
with sess.as_default():
    for i in range(100):
        batch = mnist_data.train.next_batch(50)
        train_step.run(feed_dict={img: batch[0],
                                  labels: batch[1],
                                  K.learning_phase(): 1})

acc_value = accuracy(labels, preds)
with sess.as_default():
    print (acc_value.eval(feed_dict={img: mnist_data.test.images,
                                    labels: mnist_data.test.labels,
                                    K.learning_phase(): 0}))

[ 1.  1.  1. ...,  0.  0.  1.]


!! *Nexts codes snipets are here to explain the usage. But if you execute them you need to include theses :*

In [10]:
from keras.layers import LSTM
import tensorflow as tf

### Compatibility with name scopes and device scopes
<br>
Keras layers and models are fully compatible with TensorFlow name scopes. For instance, consider the following code snippet:

In [11]:
x = tf.placeholder(tf.float32, shape=(None, 20, 64))
with tf.name_scope('block1'):
    y = LSTM(32, name='mylstm')(x)

The weights of our new LSTM layer will now be named block1/mylstm_W_i, block1/mylstm_U_i, and so on...

Similarly, devices scopes would work like this :


In [12]:
# if you use the gpu version of Tensorflow use this '/gpu:0' instead of this
with tf.device('/cpu:0'):
    x = tf.placeholder(tf.float32, shape=(None, 20, 64))
    y = LSTM(32)(x)


### Compatibility with graph scopes

Any Keras layer or model that you define inside a TensorFlow graph scope will be created as part of the specified graph, with all he's variables an operations.

In [13]:
from keras.layers import LSTM
import tensorflow as tf

my_graph = tf.Graph()
with my_graph.as_default():
    x = tf.placeholder(tf.float32, shape=(None, 20, 64))
    # LSTM operators and variables are part of the Graph
    y = LSTM(32)(x)

### Compatibility with variables scopes

Variables sharing should be done via calling multiple instance of the same Keras layer, because the TenorFlow variable scope will have NO effect on a Keras model or layer.<br>
More informations on wheight sharing with Keras could be found on the "Wheight Sharing" section of the functional API guide.<br><br>

Here is an exemple of weight sharing by reusing the same layer instance :

In [14]:
# Instantiate a Keras layer
lstm = LSTM(32)

# Instantiate 2 TensorFlow placeholders
x = tf.placeholder(tf.float32, shape=(None, 20, 64))
y = tf.placeholder(tf.float32, shape=(None, 20, 64))

# Encoding the 2 tensors with the same LSTM weights
x_encoded = lstm(x)
y_encoded = lstm(y)

### Colleting trainable weights and state updates

When using keras as a simplified interface to tensorflow, some Keras layers (stateful RNNs and BatchNormalization layers) have internal updates that need to be run as part of each training step. There are stored as a list of tensor tuples, layer.updates. We should generate assign operators for those, to be run at each training step.<br>Here's an example:

In [15]:
from keras.layers import BatchNormalization

# Instanciate a layer
layer = BatchNormalization()

# Call a layer instance
blah = layer(x)

# Generate assign operators
update_ops = []
for assign_op in layer.updates:
    update_ops.append(assign_op)

Note that if you are using a Keras Model instead of a layer, model.updates will behaves in the same way, and collects the updates of all the underlaying layers in the model.<br><br>
In addition, if you want to explicitly collect a layer's trainable weights, you can do so using "layer.trainable_weights" (or "model.trainable_weights"), a list of TensorFlow Variable instances :

In [16]:
from keras.layers import Dense

# Instanciate a layer
layer = Dense(32)

# Call a layer instance
blah = layer(x)

# List of TensorFlow Variables
print(layer.trainable_weights)

[<tf.Variable 'dense_7/kernel:0' shape=(64, 32) dtype=float32_ref>, <tf.Variable 'dense_7/bias:0' shape=(32,) dtype=float32_ref>]


Knowing this allows you to implement your own training routine based on a TensorFlow optimizer.

## Part Two : Using Keras models with TensorFlow


### Converting Keras Sequential model for use in a TensorFlow workflow

If you want to reuse a Keras Sequential model in another TensorFlow project, here is how to proceed :<br><br>

First of all, please note that, if yout pre-trained weights include convolutions that were trained ith Theano, you will need to flip the convolution kernels when loadin the weights. Theano and TensorFlow are implementing convolution in a different way<br><br>

Let's say that you are starting from the following Keras model. You want to modify it, so it takes as input a specific TensorFlow tensor : my_input_tensor.<br>
This input tensor could be a data feeder operator, for instance, or the output of a previous TensorFlow model.

In [17]:
from keras.layers import InputLayer
from keras.models import Sequential

# this is our initial Keras model
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=784))
model.add(Dense(10, activation='softmax'))

To build a Sequential model on top of a custom TensorFlow placeholder, use "keras.layers.InputLayer", then build the rest of the model.

In [18]:
from keras.layers import InputLayer

# Instanciate a layer for the input
layer = Dense(32)

# Call a layer instance for the input
blah = layer(x)

# this is the modified Keras model
model = Sequential()
model.add(InputLayer(input_tensor=blah,
                    input_shape=(None, 784)))

# and this is the rest of the model
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

That can be usefull to know how to save and load the weights or models<br>
first the weights only can be saved this way :

In [19]:
model.save_weights('my_model_weights.h5')

To load this file , you build your model, then :

In [20]:
model.load_weights('my_model_weights.h5')

Another saving technique "model.save(filepath)" will save the architecture of the model (allowing to re-create it), the weights of the model, the training configuration, and the state of the optimizer (allowing to resume training exactly where you left it) :

In [21]:
# this is only for getting the current directory for saving the file
import os
filepath = os.path.abspath(os.path.curdir)

model.save(filepath + '\\my_full_model.h5')

To load this saved model, you would use the following :

In [ ]:
from keras.models import load_model

new_model = load_model(filepath + '\\my_full_model.h5')

The last one is for saving the architecture of the model, using Json. with "model.to_json()" method that create a string. The string can be loaded by calling the "model_from_json(string)" method

Going back to our model, we can now collecting the Sequential model's output tensor :

In [23]:
output_tensor = model.output

You can now add new TensorFlow operators on top of output_tensor, and so on.

### Calling a Keras model on a TensorFlow tensor

A Keras model acts like a layer, and thus can be called on TensorFlow tensors:

In [24]:
from keras.models import Sequential

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=784))
model.add(Dense(10, activation='softmax'))

# this works! 
x = tf.placeholder(tf.float32, shape=(None, 784))
y = model(x)